In [2]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv('schufa_bug_v2.csv', sep=';')

In [7]:
from pandas import json_normalize
import json

def extract_scorebereich(json_str):
    try:
        data = json.loads(json_str)
        reaktion = data['data']['reaktion'][0]

        # Try first structure
        if 'nachmeldung' in reaktion and reaktion['nachmeldung'] and \
           'scoreinformationen' in reaktion['nachmeldung']:
            scoreinfos = reaktion['nachmeldung']['scoreinformationen']
            if scoreinfos:
                return scoreinfos[0].get('scorebereich')

        # Try bonitaetsauskunft directly
        if 'bonitaetsauskunft' in reaktion and \
           'scoreinformationen' in reaktion['bonitaetsauskunft']:
            scoreinfos = reaktion['bonitaetsauskunft']['scoreinformationen']
            if scoreinfos:
                return scoreinfos[0].get('scorebereich')

    except Exception as e:
        return None

In [8]:
data['scorebereich'] = data['response_raw_value'].apply(extract_scorebereich)

In [20]:
mismatch = data[data['rating'] != data['scorebereich']]

In [22]:
to_share = mismatch.drop('response_raw_value', axis=1)

In [24]:
to_share.to_csv('schufa_rating_mismatch.csv')